# Tuning Neural Networks with Normalization - Lab 

## Introduction

In this lab you'll build a neural network to perform a regression task.

It is worth noting that getting regression to work with neural networks can be comparatively difficult because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$), and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case for tinkering with normalization and optimization strategies to ensure proper convergence!

## Objectives

In this lab you will: 

- Fit a neural network to normalized data 
- Implement and observe the impact of various initialization techniques 
- Implement and observe the impact of various optimization techniques 

## Load the data 

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [1]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras import optimizers
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In this lab, you'll be working with the housing prices data you saw in an earlier section. However, we did a lot of preprocessing for you so you can focus on normalizing numeric features and building neural network models! The following preprocessing steps were taken (all the code can be found in the `data_preprocessing.ipynb` notebook in this repository): 

- The data was split into the training, validate, and test sets 
- All the missing values in numeric columns were replaced by the median of those columns 
- All the missing values in catetgorical columns were replaced with the word 'missing' 
- All the categorical columns were one-hot encoded 

Run the following cells to import the train, validate, and test sets:  

In [2]:
# Load all numeric features
X_train_numeric = pd.read_csv('data/X_train_numeric.csv')
X_test_numeric = pd.read_csv('data/X_test_numeric.csv')
X_val_numeric = pd.read_csv('data/X_val_numeric.csv')

# Load all categorical features
X_train_cat = pd.read_csv('data/X_train_cat.csv')
X_test_cat = pd.read_csv('data/X_test_cat.csv')
X_val_cat = pd.read_csv('data/X_val_cat.csv')

# Load all targets
y_train = pd.read_csv('data/y_train.csv')
y_test = pd.read_csv('data/y_test.csv')
y_val = pd.read_csv('data/y_val.csv')

In [4]:
# Combine all features
X_train = pd.concat([X_train_numeric, X_train_cat], axis=1)
X_val = pd.concat([X_val_numeric, X_val_cat], axis=1)
X_test = pd.concat([X_test_numeric, X_test_cat], axis=1)

As a refresher, preview the training data: 

In [5]:
# Preview the data
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,80.0,69.0,21453.0,6.0,5.0,1969.0,1969.0,0.0,938.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,60.0,79.0,12420.0,7.0,5.0,2001.0,2001.0,0.0,666.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,20.0,75.0,9742.0,8.0,5.0,2002.0,2002.0,281.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,120.0,39.0,5389.0,8.0,5.0,1995.0,1996.0,0.0,1180.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,60.0,85.0,11003.0,10.0,5.0,2008.0,2008.0,160.0,765.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


## Build a Baseline Model

Building a naive baseline model to compare performance against is a helpful reference point. From there, you can then observe the impact of various tunning procedures which will iteratively improve your model. So, let's do just that! 

In the cell below: 

- Add an input layer with `n_features` units 
- Add two hidden layers, one with 100 and the other with 50 units (make sure you use the `'relu'` activation function) 
- Add an output layer with 1 unit and `'linear'` activation 
- Compile and fit the model 

Here, we're calling .shape on our training data so that we can use the result as n_features, so we know how big to make our input layer.

In [6]:
# How big input layer?
n_features = (X_train.shape[1],)
print(n_features)

(296,)


Create your baseline model. Yo will notice is exihibits strange behavior.

*Note:* When you run this model or other models later on, you may get a comment from tf letting you about optimizing your GPU

In [7]:
# Baseline model
np.random.seed(123)
baseline_model = Sequential()

# Hidden layer with 100 units
baseline_model.add(layers.Dense(100, activation='relu', input_shape= n_features))

# Hidden layer with 50 units
baseline_model.add(layers.Dense(50, activation='relu'))

# Output layer
baseline_model.add(layers.Dense(1, activation='linear'))

# Compile the model
baseline_model.compile(optimizer='SGD', 
                       loss='mse', 
                       metrics=['mse'])

# Train the model
baseline_model.fit(X_train, 
                   y_train, 
                   batch_size=32, 
                   epochs=150, 
                   validation_data=(X_val, y_val))

Epoch 1/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 781us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - ETA: 0s - loss: nan - mse: na - 0s 761us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 765us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 843us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 782us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 776us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 782us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Ep

Epoch 68/150
33/33 [==============================] - 0s 780us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 69/150
33/33 [==============================] - 0s 707us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
33/33 [==============================] - 0s 747us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] - 0s 767us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
33/33 [==============================] - 0s 817us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
33/33 [==============================] - 0s 762us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150
33/33 [==============================] - 0s 765us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 75/150
33/33 [==============================] - 0s 783us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 76/150
33/33 [====

Epoch 135/150
33/33 [==============================] - 0s 809us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 136/150
33/33 [==============================] - 0s 806us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 137/150
33/33 [==============================] - 0s 806us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
33/33 [==============================] - 0s 840us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [==============================] - 0s 793us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 140/150
33/33 [==============================] - 0s 792us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 141/150
33/33 [==============================] - 0s 745us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 142/150
33/33 [==============================] - 0s 761us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 143/150
33

> _**Notice this extremely problematic behavior: all the values for training and validation loss are "nan". This indicates that the algorithm did not converge. The first solution to this is to normalize the input. From there, if convergence is not achieved, normalizing the output may also be required.**_ 

## Normalize the Input Data 

It's now time to normalize the input data. In the cell below: 

- Assign the column names of all numeric columns to `numeric_columns` 
- Instantiate a `StandardScaler` 
- Fit and transform `X_train_numeric`. Make sure you convert the result into a DataFrame (use `numeric_columns` as the column names) 
- Transform validate and test sets (`X_val_numeric` and `X_test_numeric`), and convert these results into DataFrames as well 
- Use the provided to combine the scaled numerical and categorical features 

In [9]:
# Numeric column names
numeric_columns = X_train_numeric.columns 

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform train data
X_train_scaled = pd.DataFrame(ss_X.fit_transform(X_train_numeric), columns=numeric_columns)

# Transform validate and test data
X_val_scaled = pd.DataFrame(ss_X.transform(X_val_numeric), columns=numeric_columns)
X_test_scaled = pd.DataFrame(ss_X.transform(X_test_numeric), columns=numeric_columns)

# Combine the scaled numerical features and categorical features
X_train = pd.concat([X_train_scaled, X_train_cat], axis=1)
X_val = pd.concat([X_val_scaled, X_val_cat], axis=1)
X_test = pd.concat([X_test_scaled, X_test_cat], axis=1)

Now run the following cell to compile a neural network model (with the same architecture as before): 

In [10]:
# Model with all normalized inputs
np.random.seed(123)
normalized_input_model = Sequential()
normalized_input_model.add(layers.Dense(100, activation='relu', input_shape=n_features))
normalized_input_model.add(layers.Dense(50, activation='relu'))
normalized_input_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_input_model.compile(optimizer='SGD', 
                               loss='mse', 
                               metrics=['mse'])

In the cell below: 
- Train the `normalized_input_model` on normalized input (`X_train`) and output (`y_train`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` argument as `(X_val, y_val)` 
Again, you may get some strange behavior.

In [11]:
# Train the model
normalized_input_model.fit(X_train, y_train, batch_size=32, epochs=150, validation_data=(X_val, y_val))

Epoch 1/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 739us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 762us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 767us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 750us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 719us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 763us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 809us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [===============

33/33 [==============================] - 0s 763us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 69/150
33/33 [==============================] - 0s 731us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
33/33 [==============================] - 0s 715us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] - 0s 731us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
33/33 [==============================] - 0s 809us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
33/33 [==============================] - 0s 869us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150
33/33 [==============================] - 0s 838us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 75/150
33/33 [==============================] - 0s 822us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 76/150
33/33 [=================

Epoch 135/150
33/33 [==============================] - 0s 748us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 136/150
33/33 [==============================] - 0s 780us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 137/150
33/33 [==============================] - 0s 686us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
33/33 [==============================] - 0s 736us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [==============================] - 0s 761us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 140/150
33/33 [==============================] - 0s 760us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 141/150
33/33 [==============================] - 0s 733us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 142/150
33/33 [==============================] - 0s 734us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 143/150
33

> _**Note that you still haven't achieved convergence! From here, it's time to normalize the output data.**_

## Normalizing the output

Again, use `StandardScaler()` to: 

- Fit and transform `y_train` 
- Transform `y_val` and `y_test` 

In [12]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform train labels
y_train_scaled = ss_y.fit_transform(y_train)

# Transform validate and test labels
y_val_scaled = ss_y.fit_transform(y_val)
y_test_scaled = ss_y.fit_transform(y_test)

In the cell below: 
- Train the `normalized_model` on normalized input (`X_train`) and output (`y_train_scaled`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` as `(X_val, y_val_scaled)` 

In [13]:
# Model with all normalized inputs and outputs
np.random.seed(123)
normalized_model = Sequential()
normalized_model.add(layers.Dense(100, activation='relu', input_shape=(n_features)))
normalized_model.add(layers.Dense(50, activation='relu'))
normalized_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model
normalized_model.fit(X_train, y_train_scaled, batch_size=32, epochs=150, validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 3ms/step - loss: 0.4651 - mse: 0.4651 - val_loss: 0.2263 - val_mse: 0.2263
Epoch 2/150
33/33 [==============================] - 0s 781us/step - loss: 0.2489 - mse: 0.2489 - val_loss: 0.1954 - val_mse: 0.1954
Epoch 3/150
33/33 [==============================] - 0s 785us/step - loss: 0.2005 - mse: 0.2005 - val_loss: 0.1799 - val_mse: 0.1799
Epoch 4/150
33/33 [==============================] - 0s 816us/step - loss: 0.1655 - mse: 0.1655 - val_loss: 0.1584 - val_mse: 0.1584
Epoch 5/150
33/33 [==============================] - 0s 784us/step - loss: 0.1522 - mse: 0.1522 - val_loss: 0.1673 - val_mse: 0.1673
Epoch 6/150
33/33 [==============================] - 0s 779us/step - loss: 0.1362 - mse: 0.1362 - val_loss: 0.1447 - val_mse: 0.1447
Epoch 7/150
33/33 [==============================] - 0s 835us/step - loss: 0.1230 - mse: 0.1230 - val_loss: 0.1393 - val_mse: 0.1393
Epoch 8/150
33/33 [==============================] - 0s 794us/step - lo

33/33 [==============================] - 0s 749us/step - loss: 0.0269 - mse: 0.0269 - val_loss: 0.1395 - val_mse: 0.1395
Epoch 61/150
33/33 [==============================] - 0s 748us/step - loss: 0.0267 - mse: 0.0267 - val_loss: 0.1343 - val_mse: 0.1343
Epoch 62/150
33/33 [==============================] - 0s 801us/step - loss: 0.0259 - mse: 0.0259 - val_loss: 0.1215 - val_mse: 0.1215
Epoch 63/150
33/33 [==============================] - 0s 846us/step - loss: 0.0254 - mse: 0.0254 - val_loss: 0.1321 - val_mse: 0.1321
Epoch 64/150
33/33 [==============================] - 0s 859us/step - loss: 0.0251 - mse: 0.0251 - val_loss: 0.1331 - val_mse: 0.1331
Epoch 65/150
33/33 [==============================] - 0s 792us/step - loss: 0.0248 - mse: 0.0248 - val_loss: 0.1322 - val_mse: 0.1322
Epoch 66/150
33/33 [==============================] - 0s 807us/step - loss: 0.0244 - mse: 0.0244 - val_loss: 0.1310 - val_mse: 0.1310
Epoch 67/150
33/33 [==============================] - 0s 801us/step - loss:

33/33 [==============================] - 0s 800us/step - loss: 0.0131 - mse: 0.0131 - val_loss: 0.1319 - val_mse: 0.1319
Epoch 122/150
33/33 [==============================] - 0s 749us/step - loss: 0.0129 - mse: 0.0129 - val_loss: 0.1272 - val_mse: 0.1272
Epoch 123/150
33/33 [==============================] - 0s 781us/step - loss: 0.0127 - mse: 0.0127 - val_loss: 0.1238 - val_mse: 0.1238
Epoch 124/150
33/33 [==============================] - 0s 780us/step - loss: 0.0129 - mse: 0.0129 - val_loss: 0.1264 - val_mse: 0.1264
Epoch 125/150
33/33 [==============================] - 0s 722us/step - loss: 0.0127 - mse: 0.0127 - val_loss: 0.1223 - val_mse: 0.1223
Epoch 126/150
33/33 [==============================] - 0s 720us/step - loss: 0.0123 - mse: 0.0123 - val_loss: 0.1272 - val_mse: 0.1272
Epoch 127/150
33/33 [==============================] - 0s 825us/step - loss: 0.0124 - mse: 0.0124 - val_loss: 0.1236 - val_mse: 0.1236
Epoch 128/150
33/33 [==============================] - 0s 885us/step 

Nicely done! After normalizing both the input and output, the model finally converged. 

- Evaluate the model (`normalized_model`) on training data (`X_train` and `y_train_scaled`) 

In [14]:
# Evaluate the model on training data
normalized_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 335us/step - loss: 0.0095 - mse: 0.0095


[0.009461627341806889, 0.009461627341806889]

- Evaluate the model (`normalized_model`) on validate data (`X_val` and `y_val_scaled`) 

In [15]:
# Evaluate the model on validate data
normalized_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 445us/step - loss: 0.1281 - mse: 0.1281


[0.1281331181526184, 0.1281331181526184]

Since the output is normalized, the metric above is not interpretable. To remedy this: 

- Generate predictions on validate data (`X_val`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_val` and `y_val_pred` 

In [16]:
# Generate predictions on validate data
y_val_pred_scaled = normalized_model.predict(X_val)

# Transform the predictions back to original scale
y_val_pred = ss_y.inverse_transform(y_val_pred_scaled)

# RMSE of validate data
np.sqrt(mean_squared_error(y_val, y_val_pred))

30243.799532463134


Great. Now that you have a converged model, you can also experiment with alternative optimizers and initialization strategies to see if you can find a better global minimum. (After all, the current models may have converged to a local minimum.) 

## Using Weight Initializers

In this section you will to use alternative initialization and optimization strategies. At the end, you'll then be asked to select the model which you believe performs the best.  

##  He Initialization

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='he_normal'` 
  
[Documentation on the He Normal Initializer](https://www.tensorflow.org/api_docs/python/tf/keras/initializers/HeNormal)

In [17]:
np.random.seed(123)
he_model = Sequential()

# Add the first hidden layer
he_model.add(layers.Dense(100, activation='relu', input_shape=n_features))

# Add another hidden layer
he_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
he_model.add(layers.Dense(1, activation='linear'))

# Compile the model
he_model.compile(optimizer='SGD', loss='mse', metrics=['mse'])

# Train the model
he_model.fit(X_train, y_train_scaled, batch_size=32, epochs=150, validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 3ms/step - loss: 0.5275 - mse: 0.5275 - val_loss: 0.2972 - val_mse: 0.2972
Epoch 2/150
33/33 [==============================] - 0s 808us/step - loss: 0.2550 - mse: 0.2550 - val_loss: 0.2425 - val_mse: 0.2425
Epoch 3/150
33/33 [==============================] - 0s 868us/step - loss: 0.1957 - mse: 0.1957 - val_loss: 0.1904 - val_mse: 0.1904
Epoch 4/150
33/33 [==============================] - 0s 840us/step - loss: 0.1652 - mse: 0.1652 - val_loss: 0.1946 - val_mse: 0.1946
Epoch 5/150
33/33 [==============================] - 0s 782us/step - loss: 0.1457 - mse: 0.1457 - val_loss: 0.1787 - val_mse: 0.1787
Epoch 6/150
33/33 [==============================] - 0s 793us/step - loss: 0.1306 - mse: 0.1306 - val_loss: 0.1728 - val_mse: 0.1728
Epoch 7/150
33/33 [==============================] - 0s 806us/step - loss: 0.1177 - mse: 0.1177 - val_loss: 0.1745 - val_mse: 0.1745
Epoch 8/150
33/33 [==============================] - 0s 855us/step - lo

33/33 [==============================] - 0s 870us/step - loss: 0.0237 - mse: 0.0237 - val_loss: 0.1729 - val_mse: 0.1729
Epoch 63/150
33/33 [==============================] - 0s 852us/step - loss: 0.0231 - mse: 0.0231 - val_loss: 0.1728 - val_mse: 0.1728
Epoch 64/150
33/33 [==============================] - 0s 809us/step - loss: 0.0226 - mse: 0.0226 - val_loss: 0.1801 - val_mse: 0.1801
Epoch 65/150
33/33 [==============================] - 0s 776us/step - loss: 0.0221 - mse: 0.0221 - val_loss: 0.1852 - val_mse: 0.1852
Epoch 66/150
33/33 [==============================] - 0s 762us/step - loss: 0.0224 - mse: 0.0224 - val_loss: 0.1730 - val_mse: 0.1730
Epoch 67/150
33/33 [==============================] - 0s 764us/step - loss: 0.0220 - mse: 0.0220 - val_loss: 0.1786 - val_mse: 0.1786
Epoch 68/150
33/33 [==============================] - 0s 794us/step - loss: 0.0215 - mse: 0.0215 - val_loss: 0.1731 - val_mse: 0.1731
Epoch 69/150
33/33 [==============================] - 0s 764us/step - loss:

33/33 [==============================] - 0s 792us/step - loss: 0.0109 - mse: 0.0109 - val_loss: 0.1801 - val_mse: 0.1801
Epoch 124/150
33/33 [==============================] - 0s 778us/step - loss: 0.0107 - mse: 0.0107 - val_loss: 0.1861 - val_mse: 0.1861
Epoch 125/150
33/33 [==============================] - 0s 807us/step - loss: 0.0106 - mse: 0.0106 - val_loss: 0.1793 - val_mse: 0.1793
Epoch 126/150
33/33 [==============================] - 0s 808us/step - loss: 0.0106 - mse: 0.0106 - val_loss: 0.1846 - val_mse: 0.1846
Epoch 127/150
33/33 [==============================] - 0s 808us/step - loss: 0.0105 - mse: 0.0105 - val_loss: 0.1725 - val_mse: 0.1725
Epoch 128/150
33/33 [==============================] - 0s 778us/step - loss: 0.0102 - mse: 0.0102 - val_loss: 0.1791 - val_mse: 0.1791
Epoch 129/150
33/33 [==============================] - 0s 856us/step - loss: 0.0102 - mse: 0.0102 - val_loss: 0.1835 - val_mse: 0.1835
Epoch 130/150
33/33 [==============================] - 0s 868us/step 

Evaluate the model (`he_model`) on training data (`X_train` and `y_train_scaled`) 

In [18]:
# Evaluate the model on training data
he_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 373us/step - loss: 0.0094 - mse: 0.0094


[0.009443357586860657, 0.009443357586860657]

Evaluate the model (`he_model`) on validate data (`X_val` and `y_val_scaled`)

In [19]:
# Evaluate the model on validate data
he_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 391us/step - loss: 0.1922 - mse: 0.1922


[0.19215868413448334, 0.19215868413448334]

## Lecun Initialization 

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='lecun_normal'` 
  
[Documentation on the Lecun Normal Initializer](https://www.tensorflow.org/api_docs/python/tf/keras/initializers/LecunNormal)

In [20]:
np.random.seed(123)
lecun_model = Sequential()

# Add the first hidden layer
lecun_model.add(layers.Dense(100, kernel_initializer='lecun_normal', activation='relu', input_shape=n_features))

# Add another hidden layer
lecun_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
lecun_model.add(layers.Dense(1, activation='linear'))

# Compile the model
lecun_model.compile(optimizer='SGD', loss='mse', metrics=['mse'])

# Train the model
lecun_model.fit(X_train, 
                y_train_scaled, 
                batch_size=32, 
                epochs=150, 
                validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 3ms/step - loss: 0.5078 - mse: 0.5078 - val_loss: 0.2194 - val_mse: 0.2194
Epoch 2/150
33/33 [==============================] - 0s 823us/step - loss: 0.2476 - mse: 0.2476 - val_loss: 0.1685 - val_mse: 0.1685
Epoch 3/150
33/33 [==============================] - 0s 841us/step - loss: 0.1965 - mse: 0.1965 - val_loss: 0.1431 - val_mse: 0.1431
Epoch 4/150
33/33 [==============================] - 0s 808us/step - loss: 0.1695 - mse: 0.1695 - val_loss: 0.1482 - val_mse: 0.1482
Epoch 5/150
33/33 [==============================] - 0s 781us/step - loss: 0.1477 - mse: 0.1477 - val_loss: 0.1480 - val_mse: 0.1480
Epoch 6/150
33/33 [==============================] - 0s 811us/step - loss: 0.1302 - mse: 0.1302 - val_loss: 0.1521 - val_mse: 0.1521
Epoch 7/150
33/33 [==============================] - 0s 825us/step - loss: 0.1176 - mse: 0.1176 - val_loss: 0.1285 - val_mse: 0.1285
Epoch 8/150
33/33 [==============================] - 0s 761us/step - lo

33/33 [==============================] - 0s 824us/step - loss: 0.0207 - mse: 0.0207 - val_loss: 0.1689 - val_mse: 0.1689
Epoch 63/150
33/33 [==============================] - 0s 808us/step - loss: 0.0205 - mse: 0.0205 - val_loss: 0.1717 - val_mse: 0.1717
Epoch 64/150
33/33 [==============================] - 0s 749us/step - loss: 0.0200 - mse: 0.0200 - val_loss: 0.1706 - val_mse: 0.1706
Epoch 65/150
33/33 [==============================] - 0s 761us/step - loss: 0.0203 - mse: 0.0203 - val_loss: 0.1681 - val_mse: 0.1681
Epoch 66/150
33/33 [==============================] - 0s 874us/step - loss: 0.0196 - mse: 0.0196 - val_loss: 0.1777 - val_mse: 0.1777
Epoch 67/150
33/33 [==============================] - 0s 807us/step - loss: 0.0194 - mse: 0.0194 - val_loss: 0.1793 - val_mse: 0.1793
Epoch 68/150
33/33 [==============================] - 0s 826us/step - loss: 0.0190 - mse: 0.0190 - val_loss: 0.1683 - val_mse: 0.1683
Epoch 69/150
33/33 [==============================] - 0s 778us/step - loss:

33/33 [==============================] - 0s 777us/step - loss: 0.0098 - mse: 0.0098 - val_loss: 0.1887 - val_mse: 0.1887
Epoch 124/150
33/33 [==============================] - 0s 776us/step - loss: 0.0093 - mse: 0.0093 - val_loss: 0.1945 - val_mse: 0.1945
Epoch 125/150
33/33 [==============================] - 0s 748us/step - loss: 0.0094 - mse: 0.0094 - val_loss: 0.1962 - val_mse: 0.1962
Epoch 126/150
33/33 [==============================] - 0s 792us/step - loss: 0.0094 - mse: 0.0094 - val_loss: 0.1928 - val_mse: 0.1928
Epoch 127/150
33/33 [==============================] - 0s 747us/step - loss: 0.0093 - mse: 0.0093 - val_loss: 0.1919 - val_mse: 0.1919
Epoch 128/150
33/33 [==============================] - 0s 730us/step - loss: 0.0092 - mse: 0.0092 - val_loss: 0.2014 - val_mse: 0.2014
Epoch 129/150
33/33 [==============================] - 0s 763us/step - loss: 0.0091 - mse: 0.0091 - val_loss: 0.2146 - val_mse: 0.2146
Epoch 130/150
33/33 [==============================] - 0s 840us/step 

Evaluate the model (`lecun_model`) on training data (`X_train` and `y_train_scaled`) 

In [21]:
# Evaluate the model on training data
lecun_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 412us/step - loss: 0.0072 - mse: 0.0072


[0.007236674893647432, 0.007236674893647432]

Evaluate the model (`lecun_model`) on validate data (`X_train` and `y_train_scaled`) 

In [22]:
# Evaluate the model on validate data
lecun_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 389us/step - loss: 0.1932 - mse: 0.1932


[0.19318783283233643, 0.19318783283233643]

Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impact of various optimization algorithms.

## RMSprop 

Compile the `rmsprop_model` with: 

- `'rmsprop'` as the optimizer 
- track `'mse'` as the loss and metric  

[Documentation on the RMS Prop Optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/experimental/RMSprop)

In [23]:
np.random.seed(123)
rmsprop_model = Sequential()
rmsprop_model.add(layers.Dense(100, activation='relu', input_shape=n_features))
rmsprop_model.add(layers.Dense(50, activation='relu'))
rmsprop_model.add(layers.Dense(1, activation='linear'))

# Compile the model
rmsprop_model.compile(optimizer='rmsprop', loss='mse', metrics=['mse'])

# Train the model
rmsprop_model.fit(X_train, 
                  y_train_scaled, 
                  batch_size=32, 
                  epochs=150, 
                  validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 3ms/step - loss: 0.3266 - mse: 0.3266 - val_loss: 0.1396 - val_mse: 0.1396
Epoch 2/150
33/33 [==============================] - 0s 852us/step - loss: 0.1711 - mse: 0.1711 - val_loss: 0.1223 - val_mse: 0.1223
Epoch 3/150
33/33 [==============================] - 0s 822us/step - loss: 0.1211 - mse: 0.1211 - val_loss: 0.2534 - val_mse: 0.2534
Epoch 4/150
33/33 [==============================] - 0s 855us/step - loss: 0.0918 - mse: 0.0918 - val_loss: 0.1210 - val_mse: 0.1210
Epoch 5/150
33/33 [==============================] - 0s 822us/step - loss: 0.0688 - mse: 0.0688 - val_loss: 0.1221 - val_mse: 0.1221
Epoch 6/150
33/33 [==============================] - 0s 825us/step - loss: 0.0606 - mse: 0.0606 - val_loss: 0.1155 - val_mse: 0.1155
Epoch 7/150
33/33 [==============================] - 0s 808us/step - loss: 0.0590 - mse: 0.0590 - val_loss: 0.1500 - val_mse: 0.1500
Epoch 8/150
33/33 [==============================] - 0s 883us/step - lo

33/33 [==============================] - 0s 855us/step - loss: 0.0094 - mse: 0.0094 - val_loss: 0.1306 - val_mse: 0.1306
Epoch 63/150
33/33 [==============================] - 0s 794us/step - loss: 0.0079 - mse: 0.0079 - val_loss: 0.1417 - val_mse: 0.1417
Epoch 64/150
33/33 [==============================] - 0s 828us/step - loss: 0.0102 - mse: 0.0102 - val_loss: 0.1562 - val_mse: 0.1562
Epoch 65/150
33/33 [==============================] - 0s 745us/step - loss: 0.0107 - mse: 0.0107 - val_loss: 0.1477 - val_mse: 0.1477
Epoch 66/150
33/33 [==============================] - 0s 790us/step - loss: 0.0084 - mse: 0.0084 - val_loss: 0.1311 - val_mse: 0.1311
Epoch 67/150
33/33 [==============================] - 0s 841us/step - loss: 0.0079 - mse: 0.0079 - val_loss: 0.1290 - val_mse: 0.1290
Epoch 68/150
33/33 [==============================] - 0s 853us/step - loss: 0.0113 - mse: 0.0113 - val_loss: 0.1302 - val_mse: 0.1302
Epoch 69/150
33/33 [==============================] - 0s 913us/step - loss:

Epoch 123/150
33/33 [==============================] - 0s 946us/step - loss: 0.0053 - mse: 0.0053 - val_loss: 0.1290 - val_mse: 0.1290
Epoch 124/150
33/33 [==============================] - 0s 823us/step - loss: 0.0058 - mse: 0.0058 - val_loss: 0.1272 - val_mse: 0.1272
Epoch 125/150
33/33 [==============================] - 0s 779us/step - loss: 0.0052 - mse: 0.0052 - val_loss: 0.1119 - val_mse: 0.1119
Epoch 126/150
33/33 [==============================] - 0s 794us/step - loss: 0.0052 - mse: 0.0052 - val_loss: 0.1503 - val_mse: 0.1503
Epoch 127/150
33/33 [==============================] - 0s 853us/step - loss: 0.0052 - mse: 0.0052 - val_loss: 0.1539 - val_mse: 0.1539
Epoch 128/150
33/33 [==============================] - 0s 778us/step - loss: 0.0048 - mse: 0.0048 - val_loss: 0.1259 - val_mse: 0.1259
Epoch 129/150
33/33 [==============================] - 0s 791us/step - loss: 0.0052 - mse: 0.0052 - val_loss: 0.1346 - val_mse: 0.1346
Epoch 130/150
33/33 [==============================] - 

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [24]:
# Evaluate the model on training data
rmsprop_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 365us/step - loss: 0.0044 - mse: 0.0044


[0.004449394065886736, 0.004449394065886736]

Evaluate the model (`rmsprop_model`) on training data (`X_val` and `y_val_scaled`) 

In [25]:
# Evaluate the model on validate data
rmsprop_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 390us/step - loss: 0.1185 - mse: 0.1185


[0.11847822368144989, 0.11847822368144989]

## Adam 

Compile the `adam_model` with: 

- `'Adam'` as the optimizer 
- track `'mse'` as the loss and metric

[Documentation on the Adam Optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam)

In [26]:
np.random.seed(123)
adam_model = Sequential()
adam_model.add(layers.Dense(100, activation='relu', input_shape=n_features))
adam_model.add(layers.Dense(50, activation='relu'))
adam_model.add(layers.Dense(1, activation='linear'))

# Compile the model
adam_model.compile(optimizer='Adam', loss='mse', metrics=['mse'])

# Train the model
adam_model.fit(X_train, 
               y_train_scaled, 
               batch_size=32, 
               epochs=150, 
               validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 3ms/step - loss: 0.3503 - mse: 0.3503 - val_loss: 0.1490 - val_mse: 0.1490
Epoch 2/150
33/33 [==============================] - 0s 859us/step - loss: 0.1913 - mse: 0.1913 - val_loss: 0.1258 - val_mse: 0.1258
Epoch 3/150
33/33 [==============================] - 0s 875us/step - loss: 0.1295 - mse: 0.1295 - val_loss: 0.1271 - val_mse: 0.1271
Epoch 4/150
33/33 [==============================] - 0s 809us/step - loss: 0.0795 - mse: 0.0795 - val_loss: 0.1401 - val_mse: 0.1401
Epoch 5/150
33/33 [==============================] - 0s 826us/step - loss: 0.0606 - mse: 0.0606 - val_loss: 0.1522 - val_mse: 0.1522
Epoch 6/150
33/33 [==============================] - 0s 809us/step - loss: 0.0508 - mse: 0.0508 - val_loss: 0.1258 - val_mse: 0.1258
Epoch 7/150
33/33 [==============================] - 0s 806us/step - loss: 0.0398 - mse: 0.0398 - val_loss: 0.1250 - val_mse: 0.1250
Epoch 8/150
33/33 [==============================] - 0s 822us/step - lo

Epoch 62/150
33/33 [==============================] - 0s 872us/step - loss: 8.2006e-04 - mse: 8.2006e-04 - val_loss: 0.1305 - val_mse: 0.1305
Epoch 63/150
33/33 [==============================] - 0s 810us/step - loss: 0.0011 - mse: 0.0011 - val_loss: 0.1452 - val_mse: 0.1452
Epoch 64/150
33/33 [==============================] - 0s 810us/step - loss: 0.0021 - mse: 0.0021 - val_loss: 0.1304 - val_mse: 0.1304
Epoch 65/150
33/33 [==============================] - 0s 807us/step - loss: 0.0023 - mse: 0.0023 - val_loss: 0.1332 - val_mse: 0.1332
Epoch 66/150
33/33 [==============================] - 0s 824us/step - loss: 0.0018 - mse: 0.0018 - val_loss: 0.1330 - val_mse: 0.1330
Epoch 67/150
33/33 [==============================] - 0s 859us/step - loss: 0.0016 - mse: 0.0016 - val_loss: 0.1399 - val_mse: 0.1399
Epoch 68/150
33/33 [==============================] - 0s 842us/step - loss: 0.0017 - mse: 0.0017 - val_loss: 0.1334 - val_mse: 0.1334
Epoch 69/150
33/33 [==============================] - 

Epoch 122/150
33/33 [==============================] - 0s 843us/step - loss: 2.5843e-04 - mse: 2.5843e-04 - val_loss: 0.1257 - val_mse: 0.1257
Epoch 123/150
33/33 [==============================] - 0s 793us/step - loss: 3.7829e-04 - mse: 3.7829e-04 - val_loss: 0.1229 - val_mse: 0.1229
Epoch 124/150
33/33 [==============================] - 0s 823us/step - loss: 6.3690e-04 - mse: 6.3690e-04 - val_loss: 0.1279 - val_mse: 0.1279
Epoch 125/150
33/33 [==============================] - 0s 883us/step - loss: 8.7058e-04 - mse: 8.7058e-04 - val_loss: 0.1257 - val_mse: 0.1257
Epoch 126/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0013 - mse: 0.0013 - val_loss: 0.1216 - val_mse: 0.1216
Epoch 127/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0037 - mse: 0.0037 - val_loss: 0.1313 - val_mse: 0.1313
Epoch 128/150
33/33 [==============================] - 0s 884us/step - loss: 0.0048 - mse: 0.0048 - val_loss: 0.1243 - val_mse: 0.1243
Epoch 129/150
33/33 [======

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [27]:
# Evaluate the model on training data
adam_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 381us/step - loss: 0.0015 - mse: 0.0015  


[0.0015437713591381907, 0.0015437713591381907]

Evaluate the model (`adam_model`) on training data (`X_val` and `y_val_scaled`) 

In [28]:
# Evaluate the model on validate data
adam_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 392us/step - loss: 0.1254 - mse: 0.1254


[0.12542881071567535, 0.12542881071567535]

## Select a Final Model

Now, select the model with the best performance based on the training and validation sets. Evaluate this top model using the test set!

In [29]:
# Evaluate the best model on test data
rmsprop_model.evaluate(X_test, y_test_scaled)

5/5 [==============================] - 0s 606us/step - loss: 0.0972 - mse: 0.0972


[0.09718532115221024, 0.09718532115221024]

As earlier, this metric is hard to interpret because the output is scaled. 

- Generate predictions on test data (`X_test`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_test` and `y_test_pred` 

In [31]:
# Generate predictions on test data
y_test_pred_scaled = rmsprop_model.predict(X_test)

# Transform the predictions back to original scale
y_test_pred = ss_y.inverse_transform(y_test_pred_scaled)

# MSE of test data
np.sqrt(mean_squared_error(y_test, y_test_pred))

29798.442850168805

## Summary  

In this lab, you worked to ensure your model converged properly by normalizing both the input and output. Additionally, you also investigated the impact of varying initialization and optimization routines.